# Using Deep Learning to Quantify Fibrosis in Muscle Tissue
## Compressing to NPZ Files
## By Sergei Issaev


### Import libraries

In [18]:
# load, split and scale the maps dataset ready for training
import os
from os import listdir
import numpy as np
import glob
from numpy import asarray
from numpy import vstack
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from numpy import savez_compressed
import matplotlib.pyplot as plt
import cv2
from matplotlib import pyplot
from pathlib import Path
RANDOM_STATE = 42
print('Environment Ready')

Environment Ready


In [19]:
# load all images in a directory into memory
def load_images(path, size=(256,512), visualize=False, patch=False):
    black = 0
    nonblack = 0
    src_list, tar_list = list(), list()
    # enumerate filenames in directory, assume all are images
    for filename in listdir(path):
        # load and resize the image
        pixels = load_img(path + filename, target_size=size)
        # convert to numpy array
        pixels = img_to_array(pixels)
        # split into satellite and map
        sat_img, map_img = pixels[:, :256], pixels[:, 256:]
        meaner = round(float(np.std(map_img)), 5)
        if patch==False:
            if meaner < 0.1 and black < 8:
                if visualize:
                    plt.imshow(cv2.cvtColor(sat_img.astype(np.uint8), cv2.COLOR_BGR2RGB), cmap='gray')
                    plt.axis('off')
                    plt.show()
                src_list.append(sat_img)
                tar_list.append(map_img)
                black += 1
                form = 'black'
            elif meaner < 0.1:
                if visualize:
                    plt.imshow(cv2.cvtColor(sat_img.astype(np.uint8), cv2.COLOR_BGR2RGB), cmap='gray')
                    plt.axis('off')
                    plt.show()
                black += 1
                form = 'black'
            else:
                nonblack += 1
                src_list.append(sat_img)
                tar_list.append(map_img)
                form = 'nonblack'
            print('black = ', black, 'nonblack = ', nonblack, 'current meaner is ', meaner, 'filename', filename, 'form', form)
        else:
            nonblack += 1
            src_list.append(sat_img)
            tar_list.append(map_img)
            form = 'nonblack'
    return [asarray(src_list), asarray(tar_list)]

In [20]:
impath = '/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt/'
image_names = os.listdir(impath)
zip_location = '/home/sergei/projects/def-rogertam/sergei/micro_scans/data/zippeddata_gt/regular/'
if os.path.isdir(zip_location):
    !rm -r $zip_location
!mkdir $zip_location

bens = []
for image in image_names:
    if image[:2] == "be":
        bens.append(image)

bens.append('bad_stain')
bens.append('.ipynb_checkpoints')
bens.append('dark')
bens.append('bright')
bens.append('liver')
        
print(f'bens is {bens}')
i = 0
while i < len(image_names):
    print(f'Checking {image_names[i]}')
    if image_names[i] in bens:
        print('delete this')
        image_names.remove(image_names[i])
        i = 0
        
    else: 
        i = i + 1

print(f'Image names is {image_names}')

bens is ['ben', 'bad_stain', '.ipynb_checkpoints', 'dark', 'bright', 'liver']
Checking bad_stain
delete this
Checking DMSO_87C2_Dia_PSR_X10
Checking .ipynb_checkpoints
delete this
Checking DMSO_87C2_Dia_PSR_X10
Checking DMSO87C8DiaPSRX10
Checking DMSO89C4DiaPSRX10
Checking DMSO90C4DiaPSRX10
Checking DMSO_89C5_Dia_PSR_X10
Checking ben
delete this
Checking DMSO_87C2_Dia_PSR_X10
Checking DMSO87C8DiaPSRX10
Checking DMSO89C4DiaPSRX10
Checking DMSO90C4DiaPSRX10
Checking DMSO_89C5_Dia_PSR_X10
Checking dark
delete this
Checking DMSO_87C2_Dia_PSR_X10
Checking DMSO87C8DiaPSRX10
Checking DMSO89C4DiaPSRX10
Checking DMSO90C4DiaPSRX10
Checking DMSO_89C5_Dia_PSR_X10
Checking bright
delete this
Checking DMSO_87C2_Dia_PSR_X10
Checking DMSO87C8DiaPSRX10
Checking DMSO89C4DiaPSRX10
Checking DMSO90C4DiaPSRX10
Checking DMSO_89C5_Dia_PSR_X10
Image names is ['DMSO_87C2_Dia_PSR_X10', 'DMSO87C8DiaPSRX10', 'DMSO89C4DiaPSRX10', 'DMSO90C4DiaPSRX10', 'DMSO_89C5_Dia_PSR_X10']


In [21]:

for i in range(len(image_names)):
    foldername = image_names[i]
    path = impath + '/' + foldername + '/'
    print(path)
    # load dataset
    [src_images, tar_images] = load_images(path)
    print()
    print('Loaded: ', src_images.shape, tar_images.shape)
    # save as compressed numpy array
    filename = zip_location + foldername.replace(" ", "") + "fascia" + '.npz'
    savez_compressed(filename, src_images, tar_images)
    print('Saved dataset: ', filename)

/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt//DMSO_87C2_Dia_PSR_X10/
black =  0 nonblack =  1 current meaner is  93.2611 filename 10000100000000199.png form nonblack
black =  0 nonblack =  2 current meaner is  126.67529 filename 10000100000000252.png form nonblack
black =  1 nonblack =  2 current meaner is  0.0 filename 10000100000000148.png form black
black =  2 nonblack =  2 current meaner is  0.0 filename 10000100000000336.png form black
black =  3 nonblack =  2 current meaner is  0.0 filename 10000100000000149.png form black
black =  3 nonblack =  3 current meaner is  103.20206 filename 10000100000000190.png form nonblack
black =  4 nonblack =  3 current meaner is  0.0 filename 10000100000000097.png form black
black =  4 nonblack =  4 current meaner is  97.08155 filename 10000100000000225.png form nonblack
black =  5 nonblack =  4 current meaner is  0.0 filename 10000100000000213.png form black
black =  5 nonblack =  5 current meaner is  118.95514 filename

black =  56 nonblack =  37 current meaner is  0.0 filename 10000100000000040.png form black
black =  57 nonblack =  37 current meaner is  0.0 filename 10000100000000275.png form black
black =  57 nonblack =  38 current meaner is  112.80367 filename 10000100000000114.png form nonblack
black =  57 nonblack =  39 current meaner is  117.61769 filename 10000100000000203.png form nonblack
black =  57 nonblack =  40 current meaner is  119.84521 filename 10000100000000055.png form nonblack
black =  57 nonblack =  41 current meaner is  68.7327 filename 10000100000000312.png form nonblack
black =  58 nonblack =  41 current meaner is  0.0 filename 10000100000000216.png form black
black =  59 nonblack =  41 current meaner is  0.0 filename 10000100000000369.png form black
black =  59 nonblack =  42 current meaner is  112.92635 filename 10000100000000188.png form nonblack
black =  59 nonblack =  43 current meaner is  53.15945 filename 10000100000000229.png form nonblack
black =  60 nonblack =  43 cu

black =  107 nonblack =  78 current meaner is  0.0 filename 10000100000000094.png form black
black =  107 nonblack =  79 current meaner is  116.10264 filename 10000100000000177.png form nonblack
black =  107 nonblack =  80 current meaner is  112.79957 filename 10000100000000075.png form nonblack
black =  108 nonblack =  80 current meaner is  0.0 filename 10000100000000342.png form black
black =  108 nonblack =  81 current meaner is  57.81887 filename 10000100000000111.png form nonblack
black =  108 nonblack =  82 current meaner is  83.86781 filename 10000100000000228.png form nonblack
black =  109 nonblack =  82 current meaner is  0.0 filename 10000100000000147.png form black
black =  110 nonblack =  82 current meaner is  0.0 filename 10000100000000038.png form black
black =  111 nonblack =  82 current meaner is  0.0 filename 10000100000000083.png form black
black =  112 nonblack =  82 current meaner is  0.0 filename 10000100000000052.png form black
black =  112 nonblack =  83 current 

black =  162 nonblack =  117 current meaner is  73.53484 filename 10000100000000293.png form nonblack
black =  162 nonblack =  118 current meaner is  123.41509 filename 10000100000000240.png form nonblack
black =  162 nonblack =  119 current meaner is  59.78752 filename 10000100000000121.png form nonblack
black =  162 nonblack =  120 current meaner is  126.63385 filename 10000100000000304.png form nonblack
black =  163 nonblack =  120 current meaner is  0.0 filename 10000100000000349.png form black
black =  163 nonblack =  121 current meaner is  125.96583 filename 10000100000000173.png form nonblack
black =  164 nonblack =  121 current meaner is  0.0 filename 10000100000000335.png form black
black =  165 nonblack =  121 current meaner is  0.0 filename 10000100000000002.png form black
black =  165 nonblack =  122 current meaner is  29.79049 filename 10000100000000133.png form nonblack
black =  166 nonblack =  122 current meaner is  0.0 filename 10000100000000358.png form black
black =  

black =  212 nonblack =  151 current meaner is  0.0 filename 10000100000000145.png form black
black =  212 nonblack =  152 current meaner is  117.92112 filename 10000100000000182.png form nonblack
black =  213 nonblack =  152 current meaner is  0.0 filename 10000100000000337.png form black
black =  214 nonblack =  152 current meaner is  0.0 filename 10000100000000085.png form black
black =  214 nonblack =  153 current meaner is  106.16522 filename 10000100000000169.png form nonblack
black =  214 nonblack =  154 current meaner is  32.78753 filename 10000100000000106.png form nonblack
black =  214 nonblack =  155 current meaner is  39.74788 filename 10000100000000077.png form nonblack
black =  215 nonblack =  155 current meaner is  0.0 filename 10000100000000030.png form black
black =  215 nonblack =  156 current meaner is  126.40463 filename 10000100000000172.png form nonblack

Loaded:  (164, 256, 256, 3) (164, 256, 256, 3)
Saved dataset:  /home/sergei/projects/def-rogertam/sergei/micro

...

black =  266 nonblack =  111 current meaner is  0.0 filename 10000100000000278.png form black
black =  267 nonblack =  111 current meaner is  0.0 filename 10000100000000125.png form black
black =  267 nonblack =  112 current meaner is  101.96731 filename 10000100000000089.png form nonblack
black =  267 nonblack =  113 current meaner is  125.71363 filename 10000100000000150.png form nonblack
black =  268 nonblack =  113 current meaner is  0.0 filename 10000100000000131.png form black
black =  268 nonblack =  114 current meaner is  125.09766 filename 10000100000000098.png form nonblack
black =  269 nonblack =  114 current meaner is  0.0 filename 10000100000000354.png form black
black =  270 nonblack =  114 current meaner is  0.0 filename 10000100000000384.png form black
black =  271 nonblack =  114 current meaner is  0.0 filename 10000100000000408.png form black
black =  272 nonblack =  114 current meaner is  0.0 filename 10000100000000230.png form black
black =  273 nonblack =  114 curr

black =  18 nonblack =  7 current meaner is  0.0 filename 10000100000000237.png form black
black =  18 nonblack =  8 current meaner is  124.50381 filename 10000100000000101.png form nonblack
black =  19 nonblack =  8 current meaner is  0.0 filename 10000100000000076.png form black
black =  19 nonblack =  9 current meaner is  110.17229 filename 10000100000000163.png form nonblack
black =  20 nonblack =  9 current meaner is  0.0 filename 10000100000000273.png form black
black =  20 nonblack =  10 current meaner is  86.14244 filename 10000100000000105.png form nonblack
black =  21 nonblack =  10 current meaner is  0.0 filename 10000100000000231.png form black
black =  22 nonblack =  10 current meaner is  0.0 filename 10000100000000220.png form black
black =  22 nonblack =  11 current meaner is  63.58068 filename 10000100000000004.png form nonblack
black =  23 nonblack =  11 current meaner is  0.0 filename 10000100000000022.png form black
black =  24 nonblack =  11 current meaner is  0.0 f

black =  72 nonblack =  41 current meaner is  0.0 filename 10000100000000307.png form black
black =  72 nonblack =  42 current meaner is  121.94958 filename 10000100000000153.png form nonblack
black =  73 nonblack =  42 current meaner is  0.0 filename 10000100000000272.png form black
black =  74 nonblack =  42 current meaner is  0.0 filename 10000100000000017.png form black
black =  75 nonblack =  42 current meaner is  0.0 filename 10000100000000212.png form black
black =  76 nonblack =  42 current meaner is  0.0 filename 10000100000000073.png form black
black =  76 nonblack =  43 current meaner is  46.61934 filename 10000100000000193.png form nonblack
black =  77 nonblack =  43 current meaner is  0.0 filename 10000100000000066.png form black
black =  78 nonblack =  43 current meaner is  0.0 filename 10000100000000078.png form black
black =  79 nonblack =  43 current meaner is  0.0 filename 10000100000000015.png form black
black =  80 nonblack =  43 current meaner is  0.0 filename 1000

black =  132 nonblack =  74 current meaner is  0.0 filename 10000100000000023.png form black
black =  133 nonblack =  74 current meaner is  0.0 filename 10000100000000300.png form black
black =  133 nonblack =  75 current meaner is  122.36024 filename 10000100000000160.png form nonblack
black =  134 nonblack =  75 current meaner is  0.0 filename 10000100000000014.png form black
black =  135 nonblack =  75 current meaner is  0.0 filename 10000100000000139.png form black
black =  135 nonblack =  76 current meaner is  63.83699 filename 10000100000000006.png form nonblack
black =  136 nonblack =  76 current meaner is  0.0 filename 10000100000000130.png form black
black =  137 nonblack =  76 current meaner is  0.0 filename 10000100000000294.png form black
black =  137 nonblack =  77 current meaner is  109.76822 filename 10000100000000093.png form nonblack
black =  137 nonblack =  78 current meaner is  110.45641 filename 10000100000000165.png form nonblack
black =  138 nonblack =  78 current

black =  191 nonblack =  105 current meaner is  0.0 filename 10000100000000062.png form black
black =  192 nonblack =  105 current meaner is  0.0 filename 10000100000000018.png form black
black =  193 nonblack =  105 current meaner is  0.0 filename 10000100000000219.png form black
black =  194 nonblack =  105 current meaner is  0.0 filename 10000100000000043.png form black
black =  194 nonblack =  106 current meaner is  56.90745 filename 10000100000000176.png form nonblack
black =  194 nonblack =  107 current meaner is  124.12821 filename 10000100000000145.png form nonblack
black =  195 nonblack =  107 current meaner is  0.0 filename 10000100000000182.png form black
black =  196 nonblack =  107 current meaner is  0.0 filename 10000100000000085.png form black
black =  196 nonblack =  108 current meaner is  99.07845 filename 10000100000000169.png form nonblack
black =  196 nonblack =  109 current meaner is  83.61763 filename 10000100000000106.png form nonblack
black =  197 nonblack =  10

In [22]:
impath = '/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt/dark/FR94_45D7_TA_R_3_X10.tif/'
image_names = os.listdir(impath)
zip_location = '/home/sergei/projects/def-rogertam/sergei/micro_scans/data/zippeddata_gt/dark/FR94_45D7_TA_R_3_X10.tif/'
if os.path.isdir(zip_location):
    !rm -r $zip_location
Path(zip_location).mkdir(parents=True, exist_ok=True)

print(image_names)

['Tile1', 'Tile4', 'Tile3', 'Tile2']


In [23]:

for i in range(len(image_names)):
    foldername = image_names[i]
    path = impath + '/' + foldername + '/'
    print(path)
    # load dataset
    [src_images, tar_images] = load_images(path, patch=True, visualize=True)
    print()
    print('Loaded: ', src_images.shape, tar_images.shape)
    # save as compressed numpy array
    filename = zip_location + foldername.replace(" ", "") + "fascia" + '.npz'
    savez_compressed(filename, src_images, tar_images)
    print('Saved dataset: ', filename)

/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt/dark/FR94_45D7_TA_R_3_X10.tif//Tile1/

Loaded:  (15, 256, 256, 3) (15, 256, 256, 3)
Saved dataset:  /home/sergei/projects/def-rogertam/sergei/micro_scans/data/zippeddata_gt/dark/FR94_45D7_TA_R_3_X10.tif/Tile1fascia.npz
/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt/dark/FR94_45D7_TA_R_3_X10.tif//Tile4/

Loaded:  (15, 256, 256, 3) (15, 256, 256, 3)
Saved dataset:  /home/sergei/projects/def-rogertam/sergei/micro_scans/data/zippeddata_gt/dark/FR94_45D7_TA_R_3_X10.tif/Tile4fascia.npz
/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt/dark/FR94_45D7_TA_R_3_X10.tif//Tile3/

Loaded:  (15, 256, 256, 3) (15, 256, 256, 3)
Saved dataset:  /home/sergei/projects/def-rogertam/sergei/micro_scans/data/zippeddata_gt/dark/FR94_45D7_TA_R_3_X10.tif/Tile3fascia.npz
/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt/dark/FR94_45D7_TA_R_3_X10.tif//Tile2/

Loaded:  (15, 256, 2

In [24]:
impath = '/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt/dark/FR94_45D7_TA_L_3_X10.tif/'
image_names = os.listdir(impath)
zip_location = '/home/sergei/projects/def-rogertam/sergei/micro_scans/data/zippeddata_gt/dark/FR94_45D7_TA_L_3_X10.tif/'
if os.path.isdir(zip_location):
    !rm -r $zip_location
Path(zip_location).mkdir(parents=True, exist_ok=True)

print(image_names)

['Tile1', 'Tile4', 'Tile3', 'Tile2']


In [25]:

for i in range(len(image_names)):
    foldername = image_names[i]
    path = impath + '/' + foldername + '/'
    print(path)
    # load dataset
    [src_images, tar_images] = load_images(path, patch=True)
    print()
    print('Loaded: ', src_images.shape, tar_images.shape)
    # save as compressed numpy array
    filename = zip_location + foldername.replace(" ", "") + "fascia" + '.npz'
    savez_compressed(filename, src_images, tar_images)
    print('Saved dataset: ', filename)

/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt/dark/FR94_45D7_TA_L_3_X10.tif//Tile1/

Loaded:  (15, 256, 256, 3) (15, 256, 256, 3)
Saved dataset:  /home/sergei/projects/def-rogertam/sergei/micro_scans/data/zippeddata_gt/dark/FR94_45D7_TA_L_3_X10.tif/Tile1fascia.npz
/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt/dark/FR94_45D7_TA_L_3_X10.tif//Tile4/

Loaded:  (15, 256, 256, 3) (15, 256, 256, 3)
Saved dataset:  /home/sergei/projects/def-rogertam/sergei/micro_scans/data/zippeddata_gt/dark/FR94_45D7_TA_L_3_X10.tif/Tile4fascia.npz
/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt/dark/FR94_45D7_TA_L_3_X10.tif//Tile3/

Loaded:  (15, 256, 256, 3) (15, 256, 256, 3)
Saved dataset:  /home/sergei/projects/def-rogertam/sergei/micro_scans/data/zippeddata_gt/dark/FR94_45D7_TA_L_3_X10.tif/Tile3fascia.npz
/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt/dark/FR94_45D7_TA_L_3_X10.tif//Tile2/

Loaded:  (15, 256, 2

In [26]:
impath = '/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt/bright/FR94_45D7_TA_R_4_X10.tif/'
image_names = os.listdir(impath)
zip_location = '/home/sergei/projects/def-rogertam/sergei/micro_scans/data/zippeddata_gt/bright/FR94_45D7_TA_R_4_X10.tif/'
if os.path.isdir(zip_location):
    !rm -r $zip_location
Path(zip_location).mkdir(parents=True, exist_ok=True)

print(image_names)

['Tile1', 'Tile4', 'Tile3', 'Tile2']


In [27]:

for i in range(len(image_names)):
    foldername = image_names[i]
    path = impath + '/' + foldername + '/'
    print(path)
    # load dataset
    [src_images, tar_images] = load_images(path, patch=True)
    print()
    print('Loaded: ', src_images.shape, tar_images.shape)
    # save as compressed numpy array
    filename = zip_location + foldername.replace(" ", "") + "fascia" + '.npz'
    savez_compressed(filename, src_images, tar_images)
    print('Saved dataset: ', filename)

/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt/bright/FR94_45D7_TA_R_4_X10.tif//Tile1/

Loaded:  (15, 256, 256, 3) (15, 256, 256, 3)
Saved dataset:  /home/sergei/projects/def-rogertam/sergei/micro_scans/data/zippeddata_gt/bright/FR94_45D7_TA_R_4_X10.tif/Tile1fascia.npz
/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt/bright/FR94_45D7_TA_R_4_X10.tif//Tile4/

Loaded:  (15, 256, 256, 3) (15, 256, 256, 3)
Saved dataset:  /home/sergei/projects/def-rogertam/sergei/micro_scans/data/zippeddata_gt/bright/FR94_45D7_TA_R_4_X10.tif/Tile4fascia.npz
/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt/bright/FR94_45D7_TA_R_4_X10.tif//Tile3/

Loaded:  (15, 256, 256, 3) (15, 256, 256, 3)
Saved dataset:  /home/sergei/projects/def-rogertam/sergei/micro_scans/data/zippeddata_gt/bright/FR94_45D7_TA_R_4_X10.tif/Tile3fascia.npz
/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt/bright/FR94_45D7_TA_R_4_X10.tif//Tile2/

Loaded

In [28]:
impath = '/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt/bright/FR94_45D7_TA_L_4_X10.tif/'
image_names = os.listdir(impath)
zip_location = '/home/sergei/projects/def-rogertam/sergei/micro_scans/data/zippeddata_gt/bright/FR94_45D7_TA_L_4_X10.tif/'
if os.path.isdir(zip_location):
    !rm -r $zip_location
Path(zip_location).mkdir(parents=True, exist_ok=True)

print(image_names)

['Tile1', 'Tile4', 'Tile3', 'Tile2']


In [29]:

for i in range(len(image_names)):
    foldername = image_names[i]
    path = impath + '/' + foldername + '/'
    print(path)
    # load dataset
    [src_images, tar_images] = load_images(path, patch=True)
    print()
    print('Loaded: ', src_images.shape, tar_images.shape)
    # save as compressed numpy array
    filename = zip_location + foldername.replace(" ", "") + "fascia" + '.npz'
    savez_compressed(filename, src_images, tar_images)
    print('Saved dataset: ', filename)

/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt/bright/FR94_45D7_TA_L_4_X10.tif//Tile1/

Loaded:  (15, 256, 256, 3) (15, 256, 256, 3)
Saved dataset:  /home/sergei/projects/def-rogertam/sergei/micro_scans/data/zippeddata_gt/bright/FR94_45D7_TA_L_4_X10.tif/Tile1fascia.npz
/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt/bright/FR94_45D7_TA_L_4_X10.tif//Tile4/

Loaded:  (15, 256, 256, 3) (15, 256, 256, 3)
Saved dataset:  /home/sergei/projects/def-rogertam/sergei/micro_scans/data/zippeddata_gt/bright/FR94_45D7_TA_L_4_X10.tif/Tile4fascia.npz
/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt/bright/FR94_45D7_TA_L_4_X10.tif//Tile3/

Loaded:  (15, 256, 256, 3) (15, 256, 256, 3)
Saved dataset:  /home/sergei/projects/def-rogertam/sergei/micro_scans/data/zippeddata_gt/bright/FR94_45D7_TA_L_4_X10.tif/Tile3fascia.npz
/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt/bright/FR94_45D7_TA_L_4_X10.tif//Tile2/

Loaded

In [30]:
impath = '/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt/bad_stain/5a1/'
image_names = os.listdir(impath)
zip_location = '/home/sergei/projects/def-rogertam/sergei/micro_scans/data/zippeddata_gt/bad_stain/5a1/'
if os.path.isdir(zip_location):
    !rm -r $zip_location
Path(zip_location).mkdir(parents=True, exist_ok=True)

print(image_names)

['Tile1', 'Tile4', 'Tile6', 'Tile3', 'Tile2', 'Tile5']


In [31]:

for i in range(len(image_names)):
    foldername = image_names[i]
    path = impath + '/' + foldername + '/'
    print(path)
    # load dataset
    [src_images, tar_images] = load_images(path, visualize=True)
    print()
    print('Loaded: ', src_images.shape, tar_images.shape)
    # save as compressed numpy array
    filename = zip_location + foldername.replace(" ", "") + "fascia" + '.npz'
    savez_compressed(filename, src_images, tar_images)
    print('Saved dataset: ', filename)

/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt/bad_stain/5a1//Tile1/
black =  0 nonblack =  1 current meaner is  127.42104 filename 10000100000000001.png form nonblack
black =  0 nonblack =  2 current meaner is  97.47057 filename 10000100000000000.png form nonblack

Loaded:  (2, 256, 256, 3) (2, 256, 256, 3)
Saved dataset:  /home/sergei/projects/def-rogertam/sergei/micro_scans/data/zippeddata_gt/bad_stain/5a1/Tile1fascia.npz
/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt/bad_stain/5a1//Tile4/
black =  0 nonblack =  1 current meaner is  126.50732 filename 10000100000000001.png form nonblack
black =  0 nonblack =  2 current meaner is  127.0658 filename 10000100000000000.png form nonblack

Loaded:  (2, 256, 256, 3) (2, 256, 256, 3)
Saved dataset:  /home/sergei/projects/def-rogertam/sergei/micro_scans/data/zippeddata_gt/bad_stain/5a1/Tile4fascia.npz
/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt/bad_stain/5a1//Tile6/
b

In [32]:
impath = '/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt/ben/'
image_names = os.listdir(impath)
# temp = []
# for folder in image_names:
#     if folder[-1] == "1":
#         temp.append(folder)
# print(f"Temp is {temp}")
# image_names = temp
zip_location = '/home/sergei/projects/def-rogertam/sergei/micro_scans/data/zippeddata_gt/ben/'
if os.path.isdir(zip_location):
    !rm -r $zip_location
Path(zip_location).mkdir(parents=True, exist_ok=True)

print(image_names)

['F1gt2', 'D1gt2', 'G1gt2', 'B1gt2', 'A1gt2', 'E1gt2', 'C1gt2', 'H1gt2']


In [33]:
for i in range(len(image_names)):
    foldername = image_names[i]
    path = impath + '/' + foldername + '/'
    print(path)
    # load dataset
    [src_images, tar_images] = load_images(path)
    print()
    print('Loaded: ', src_images.shape, tar_images.shape)
    # save as compressed numpy array
    filename = zip_location + foldername.replace(" ", "") + "fascia" + '.npz'
    savez_compressed(filename, src_images, tar_images)
    print('Saved dataset: ', filename)

/home/sergei/projects/def-rogertam/sergei/micro_scans/data/patchdir_gt/ben//F1gt2/
black =  1 nonblack =  0 current meaner is  0.0 filename 10000100000000004.png form black
black =  2 nonblack =  0 current meaner is  0.0 filename 10000100000000022.png form black
black =  2 nonblack =  1 current meaner is  120.11245 filename 10000100000000013.png form nonblack
black =  3 nonblack =  1 current meaner is  0.0 filename 10000100000000020.png form black
black =  3 nonblack =  2 current meaner is  39.39032 filename 10000100000000003.png form nonblack
black =  3 nonblack =  3 current meaner is  121.75064 filename 10000100000000011.png form nonblack
black =  4 nonblack =  3 current meaner is  0.0 filename 10000100000000005.png form black
black =  4 nonblack =  4 current meaner is  65.65866 filename 10000100000000017.png form nonblack
black =  4 nonblack =  5 current meaner is  99.31411 filename 10000100000000015.png form nonblack
black =  5 nonblack =  5 current meaner is  0.0 filename 10000100

black =  25 nonblack =  19 current meaner is  0.0 filename 10000100000000088.png form black
black =  25 nonblack =  20 current meaner is  127.15784 filename 10000100000000110.png form nonblack
black =  26 nonblack =  20 current meaner is  0.0 filename 10000100000000084.png form black
black =  26 nonblack =  21 current meaner is  125.94627 filename 10000100000000033.png form nonblack
black =  27 nonblack =  21 current meaner is  0.0 filename 10000100000000157.png form black
black =  28 nonblack =  21 current meaner is  0.0 filename 10000100000000143.png form black
black =  28 nonblack =  22 current meaner is  59.78019 filename 10000100000000115.png form nonblack
black =  29 nonblack =  22 current meaner is  0.0 filename 10000100000000060.png form black
black =  30 nonblack =  22 current meaner is  0.0 filename 10000100000000167.png form black
black =  30 nonblack =  23 current meaner is  126.12926 filename 10000100000000124.png form nonblack
black =  31 nonblack =  23 current meaner is 

black =  79 nonblack =  55 current meaner is  0.0 filename 10000100000000072.png form black
black =  80 nonblack =  55 current meaner is  0.0 filename 10000100000000095.png form black
black =  81 nonblack =  55 current meaner is  0.0 filename 10000100000000102.png form black
black =  82 nonblack =  55 current meaner is  0.0 filename 10000100000000156.png form black
black =  83 nonblack =  55 current meaner is  0.0 filename 10000100000000053.png form black
black =  84 nonblack =  55 current meaner is  0.0 filename 10000100000000048.png form black
black =  85 nonblack =  55 current meaner is  0.0 filename 10000100000000161.png form black
black =  85 nonblack =  56 current meaner is  127.25002 filename 10000100000000027.png form nonblack
black =  86 nonblack =  56 current meaner is  0.0 filename 10000100000000067.png form black
black =  87 nonblack =  56 current meaner is  0.0 filename 10000100000000012.png form black
black =  87 nonblack =  57 current meaner is  127.00488 filename 100001

black =  33 nonblack =  17 current meaner is  126.23137 filename 10000100000000189.png form nonblack
black =  34 nonblack =  17 current meaner is  0.0 filename 10000100000000299.png form black
black =  35 nonblack =  17 current meaner is  0.0 filename 10000100000000966.png form black
black =  36 nonblack =  17 current meaner is  0.0 filename 10000100000000671.png form black
black =  37 nonblack =  17 current meaner is  0.0 filename 10000100000000029.png form black
black =  37 nonblack =  18 current meaner is  60.93853 filename 10000100000000773.png form nonblack
black =  37 nonblack =  19 current meaner is  127.4704 filename 10000100000000232.png form nonblack
black =  38 nonblack =  19 current meaner is  0.0 filename 10000100000000527.png form black
black =  39 nonblack =  19 current meaner is  0.0 filename 10000100000000412.png form black
black =  40 nonblack =  19 current meaner is  0.0 filename 10000100000000136.png form black
black =  40 nonblack =  20 current meaner is  126.31934

black =  99 nonblack =  45 current meaner is  0.0 filename 10000100000001126.png form black
black =  100 nonblack =  45 current meaner is  0.0 filename 10000100000000013.png form black
black =  100 nonblack =  46 current meaner is  15.68821 filename 10000100000000056.png form nonblack
black =  101 nonblack =  46 current meaner is  0.0 filename 10000100000000035.png form black
black =  102 nonblack =  46 current meaner is  0.0 filename 10000100000000916.png form black
black =  103 nonblack =  46 current meaner is  0.0 filename 10000100000001102.png form black
black =  104 nonblack =  46 current meaner is  0.0 filename 10000100000000882.png form black
black =  105 nonblack =  46 current meaner is  0.0 filename 10000100000000037.png form black
black =  105 nonblack =  47 current meaner is  57.86468 filename 10000100000001083.png form nonblack
black =  106 nonblack =  47 current meaner is  0.0 filename 10000100000000215.png form black
black =  107 nonblack =  47 current meaner is  0.0 file

...

black =  622 nonblack =  162 current meaner is  112.59576 filename 10000100000000160.png form nonblack
black =  623 nonblack =  162 current meaner is  0.0 filename 10000100000000763.png form black
black =  624 nonblack =  162 current meaner is  0.0 filename 10000100000000014.png form black
black =  625 nonblack =  162 current meaner is  0.0 filename 10000100000000380.png form black
black =  626 nonblack =  162 current meaner is  0.0 filename 10000100000000871.png form black
black =  626 nonblack =  163 current meaner is  121.01295 filename 10000100000000856.png form nonblack
black =  627 nonblack =  163 current meaner is  0.0 filename 10000100000001049.png form black
black =  628 nonblack =  163 current meaner is  0.0 filename 10000100000000139.png form black
black =  629 nonblack =  163 current meaner is  0.0 filename 10000100000000521.png form black
black =  630 nonblack =  163 current meaner is  0.0 filename 10000100000000386.png form black
black =  631 nonblack =  163 current meane

black =  686 nonblack =  185 current meaner is  0.0 filename 10000100000000762.png form black
black =  687 nonblack =  185 current meaner is  0.0 filename 10000100000000240.png form black
black =  688 nonblack =  185 current meaner is  0.0 filename 10000100000000837.png form black
black =  689 nonblack =  185 current meaner is  0.0 filename 10000100000000602.png form black
black =  690 nonblack =  185 current meaner is  0.0 filename 10000100000001127.png form black
black =  690 nonblack =  186 current meaner is  120.92226 filename 10000100000000121.png form nonblack
black =  691 nonblack =  186 current meaner is  0.0 filename 10000100000000796.png form black
black =  692 nonblack =  186 current meaner is  0.0 filename 10000100000000715.png form black
black =  693 nonblack =  186 current meaner is  0.0 filename 10000100000000304.png form black
black =  694 nonblack =  186 current meaner is  0.0 filename 10000100000001091.png form black
black =  695 nonblack =  186 current meaner is  0.0

black =  759 nonblack =  203 current meaner is  0.0 filename 10000100000000979.png form black
black =  759 nonblack =  204 current meaner is  121.36776 filename 10000100000000778.png form nonblack
black =  759 nonblack =  205 current meaner is  124.91253 filename 10000100000000776.png form nonblack
black =  760 nonblack =  205 current meaner is  0.0 filename 10000100000000445.png form black
black =  761 nonblack =  205 current meaner is  0.0 filename 10000100000000250.png form black
black =  762 nonblack =  205 current meaner is  0.0 filename 10000100000000371.png form black
black =  763 nonblack =  205 current meaner is  0.0 filename 10000100000000637.png form black
black =  764 nonblack =  205 current meaner is  0.0 filename 10000100000001048.png form black
black =  765 nonblack =  205 current meaner is  0.0 filename 10000100000001081.png form black
black =  766 nonblack =  205 current meaner is  0.0 filename 10000100000000053.png form black
black =  766 nonblack =  206 current meane

black =  829 nonblack =  225 current meaner is  0.0 filename 10000100000001140.png form black
black =  830 nonblack =  225 current meaner is  0.0 filename 10000100000000758.png form black
black =  831 nonblack =  225 current meaner is  0.0 filename 10000100000001074.png form black
black =  832 nonblack =  225 current meaner is  0.0 filename 10000100000000407.png form black
black =  833 nonblack =  225 current meaner is  0.0 filename 10000100000000245.png form black
black =  834 nonblack =  225 current meaner is  0.0 filename 10000100000000443.png form black
black =  834 nonblack =  226 current meaner is  119.7032 filename 10000100000000746.png form nonblack
black =  835 nonblack =  226 current meaner is  0.0 filename 10000100000001133.png form black
black =  836 nonblack =  226 current meaner is  0.0 filename 10000100000000351.png form black
black =  837 nonblack =  226 current meaner is  0.0 filename 10000100000000764.png form black
black =  838 nonblack =  226 current meaner is  0.0 

black =  897 nonblack =  247 current meaner is  0.0 filename 10000100000001104.png form black
black =  898 nonblack =  247 current meaner is  0.0 filename 10000100000001122.png form black
black =  899 nonblack =  247 current meaner is  0.0 filename 10000100000001109.png form black
black =  900 nonblack =  247 current meaner is  0.0 filename 10000100000000106.png form black
black =  901 nonblack =  247 current meaner is  0.0 filename 10000100000000629.png form black
black =  901 nonblack =  248 current meaner is  127.2951 filename 10000100000000077.png form nonblack
black =  901 nonblack =  249 current meaner is  126.20131 filename 10000100000000738.png form nonblack
black =  901 nonblack =  250 current meaner is  115.25702 filename 10000100000000787.png form nonblack
black =  902 nonblack =  250 current meaner is  0.0 filename 10000100000000418.png form black
black =  903 nonblack =  250 current meaner is  0.0 filename 10000100000000030.png form black
black =  904 nonblack =  250 curre